# 3. Building Compositional Space Complexes to Express Elaborate Design Problems

> ***(before running this code, please consult Quick Start to make sure everything is set up)***

> You will need the same dependencies as in [Tutorial No.2](02.AdditiveManufacturingPathPlanning.ipynb) (i.e., `pycalphad` and `pathfinding`), and additionally `igraph` in order to efficiently visualize the examples presented in this tutorial. **If you are running this in `nimplex`'s Codespaces, everything has been pre-installed for you.**

**In this tutorial, we will build upon `nimplex`'s graph representations and leverage our ability to combine ("stitch") them together without affecting the way we express the problem at hand and in an effortless fashion to dramatically speed up materials exploration.** In particular, we will show how to:

1. **Plot graphs** (as in mathematical objects, not any figures) using [`igraph`](https://igraph.org/), which can be done quite directly using `nimplex`'s point grid and neighbor lists `edges = [(i,n) for i in range(len(gridAtt)) for n in nList[i]]`

2. **Identify ordered subsystems in `nimplex` grids, like `A-B-C` and `C-A-B` within `A-B-C-D-E` and `D-C-G-F-A-H-B` to establish connectivity between them.** We will use this to combine 3 4-component systems (tetrahedra) by 2 3-component subsystems (triangles) to create a chain.

3. Identify all subspaces of a given order in low dimensional and high dimensional spaces (e.g., all quantized compositions of any 3 things out of N) and combine ("stitch") them together to form a **simplex graphs that intersect themselves in 3D because of high dimensionality, yet still have the same (graph) structure**.

4. **Construct a graph complex to explore all ternary combinations of `["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]` (7 alloys)** under equilibrium phase constraint (relatively expensive to compute) and then explore the space with an additional low-cost screenig constraint (RMSAD - alloy strenght surrogate) that could also be an ML surrogate.

In [1]:
import nimplex
from utils import stitching
from IPython.display import clear_output
from itertools import combinations

In [2]:
import igraph as ig
import plotly.graph_objs as go
import numpy as np
import random
random.seed(123)

## Plotting 4-Component Simplex Graph with `igraph`

In [3]:
dim = 4
ndiv = 6
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)

In [4]:
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [5]:
def plotGraph(edges: list):
    # Lets generate some seed positions for the nodes
    G = ig.Graph(edges)
    layout = G.layout_kamada_kawai(dim=3)
    layout_array = np.array(layout.coords)
    # Create edge traces
    edge_traces = []
    for edge in G.es:
        start, end = edge.tuple
        x0, y0, z0 = layout_array[start]
        x1, y1, z1 = layout_array[end]
        width = 5
        edge_trace = go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1],
                                line=dict(width=width, color='lightgray'),
                                opacity=0.2,
                                hoverinfo='none', mode='lines')
        edge_traces.append(edge_trace)

    node_trace = go.Scatter3d(x=layout_array[:, 0], y=layout_array[:, 1], z=layout_array[:, 2],
                            mode='markers',
                            marker=dict(size=3, 
                                        opacity=0.5,
                                        line=None,
                                        color='blue'),
                            
                            text=[f"Node {i}" for i in range(len(gridAtt))],
                            hoverinfo='text')

    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])

    # Update layout
    fig.update_layout(
        title='3D Graph with Feasibility Types',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=0),
        width=800
    )

    fig.show()

In [6]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 3 4-Component Systems into A Complex

In [7]:
sys1 = ["A", "B", "C", "D"]
sys2 = ["B", "C", "D", "E"]
sys3 = ["B", "D", "E", "F"]

In [8]:
offset = len(gridAtt)
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset,n+offset))
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset*2,n+offset*2))

In [9]:
#plotGraph(edges)
clear_output()

In [10]:
stitchingPoints = stitching.findStitchingPoints_py(dim, ndiv, components=["A", "B", "C", "D"])
print(stitchingPoints["A-B"])

[83, 82, 79, 73, 63, 48, 27]


In [11]:
stitchingPoints1 = stitching.findStitchingPoints_py(dim, ndiv, components=sys1, offset=0)
stitchingPoints2 = stitching.findStitchingPoints_py(dim, ndiv, components=sys2, offset=len(gridAtt))
stitchingPoints3 = stitching.findStitchingPoints_py(dim, ndiv, components=sys3, offset=len(gridAtt)*2)

In [12]:
for i, j in zip(stitchingPoints1["B-C-D"], stitchingPoints2["B-C-D"]):
    edges.append((i, j))

In [13]:
for i, j in zip(stitchingPoints2["B-D-E"], stitchingPoints3["B-D-E"]):
    edges.append((i, j))

In [14]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 9 3-Component Systems into A Complex
### Disregard this section for now Adam

In [15]:
dim = 3
ndiv = 6
nsys = 9
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [16]:
len(gridAtt)

28

In [17]:
offset = len(gridAtt)
for sys in range(1,nsys+1):
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset*sys,n+offset*sys))

In [18]:
#plotGraph(edges)
clear_output()

In [19]:
import pandas as pd

In [20]:
from pycalphad import Database

In [21]:
dbCrNiV = Database("ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB")
dbCrFeTi = Database("ammap/databases/Cr-Fe-Ti_wang2017.tdb")
dbCrFeNi = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
dbCrNiTi= Database("ammap/databases/Cr-Ni-Ti_huang2018.tdb")
dbCrTiV= Database("ammap/databases/Cr-Ti-V_ghosh2002.tdb")
dbFeNiTi= Database("ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb")
dbFeNiV= Database("ammap/databases/Fe-Ni-V_zhao2014.tdb")
dbFeTiV= Database("ammap/databases/Fe-Ti-V_guo2012.TDB")
dbNiTiV= Database("ammap/databases/Ni-Ti-V_zou2018.tdb")
CrNiV = ['Cr', 'Ni', 'V']
CrFeTi = ['Cr', 'Fe', 'Ti']
CrFeNi = ['Cr', 'Fe', 'Ni']
CrNiTi = ['Cr', 'Ni', 'Ti']
CrTiV = ['Cr', 'Ti', 'V']
FeNiTi = ['Fe', 'Ni', 'Ti']
FeNiV = ['Fe', 'Ni', 'V']
FeTiV = ['Fe', 'Ti', 'V']
NiTiV = ['Ni', 'Ti', 'V']


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


In [22]:
stitchingPointsCrNiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiV, offset=0)
stitchingPointsCrFeTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeTi, offset=len(gridAtt))
stitchingPointsCrFeNi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeNi, offset=len(gridAtt)*2)
stitchingPointsCrNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiTi, offset=len(gridAtt)*3)
stitchingPointsCrTiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrTiV, offset=len(gridAtt)*4)
stitchingPointsFeNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiTi, offset=len(gridAtt)*5)
stitchingPointsFeNiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiV, offset=len(gridAtt)*6)
stitchingPointsFeTiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeTiV, offset=len(gridAtt)*7)
stitchingPointsNiTiV = stitching.findStitchingPoints_py(dim, ndiv, components=NiTiV, offset=len(gridAtt)*8)

## Finding and Combining 3-Component Subspaces in 4-Component Space

In [23]:
from itertools import combinations
elements = ["A", "B", "C", "D"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]


In [24]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [25]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))

In [26]:
#plotGraph(edges)
clear_output()

In [27]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1)]
('A', 'C'): occurs between ternary [(0, 2)]
('B', 'C'): occurs between ternary [(0, 3)]
('A', 'D'): occurs between ternary [(1, 2)]
('B', 'D'): occurs between ternary [(1, 3)]
('C', 'D'): occurs between ternary [(2, 3)]


In [28]:
for stitchingBinary, ternaryPair in stitchingBinaries.items():
    ternary1, ternary2 = ternaryPair[0][0], ternaryPair[0][1]
    stitching1 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary1],
        offset=ternary1*len(gridAtt)
        )["-".join(stitchingBinary)]
    stitching2 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary2],
        offset=ternary2*len(gridAtt)
        )["-".join(stitchingBinary)]
    print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
    for i, j in zip(stitching1, stitching2):
        edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 1 and 2 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [272, 270, 267, 263, 258, 252, 245, 237, 228, 218, 207, 195, 182]
Stitching 1 and 3 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 2 and 3 at ('C', 'D') from [194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182] to [285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273]


In [29]:
#plotGraph(edges)
clear_output()

## In Higher Dimensional Cases (Stretched 2D Spaces Crossing Each Other in 3D)

In [30]:
from itertools import combinations
elements = ["A", "B", "C", "D", "E"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'D', 'E'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'D', 'E'), ('C', 'D', 'E')]


In [31]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [32]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))


In [33]:
#plotGraph(edges)
clear_output()

In [34]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('A', 'C'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('B', 'C'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('A', 'D'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('B', 'D'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('A', 'E'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('B', 'E'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('C', 'D'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('C', 'E'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('D', 'E'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [35]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 1 and 2 at ('A', 'B') from [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 0 and 4 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]


Stitching 3 and 4 at ('A', 'C') from [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 0 and 6 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [636, 635, 633, 630, 626, 621, 615, 608, 600, 591, 581, 570, 558]
Stitching 0 and 7 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [727, 726, 724, 721, 717, 712, 706, 699, 691, 682, 672, 661, 649]
Stitching 6 and 7 at ('B', 'C') from [636, 635, 633, 630, 626, 621, 615, 608, 600, 591, 581, 570, 558] to [727, 726, 724, 721, 717, 712, 706, 699, 691, 682, 672, 661, 649]
Stitching 1 and 3 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 1 and 5 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [545, 544, 542, 539, 535, 530, 524, 517, 509, 500, 490, 479, 467]
Stitching 3 and 

In [36]:
#plotGraph(edges)
clear_output()

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 5-Component Space (w. Infeasibility Gliding) #THIS ONE ADAM

In [37]:
elementalSpaceComponents = ["Cr","Fe","Ni","Ti","V"]
#attainableSpaceComponents = ["SS304L", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]
attainableSpaceComponentPositions = [
    #"Cr","Fe","Ni","Ti","V"
    #[0.1993865031, 0.7044989775,0.096114519430,0,0],
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Cr', 'Fe', 'Ni')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
('Cr', 'Fe', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Fe', 'V')                        -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ni', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Ni', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ti', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ti', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Ni', 'Ti', 'V')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [38]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
compositions_with_id = []  # List to store compositions with their identifiers
ternaries_with_id = []  # New list to store ternaries_CompPos with their identifiers

# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    ternaries_with_id.append((terncp, i))  # Add terncp and its id to the new list
    
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    
    # Attach identifier to each composition
    compositions_with_id.extend([(comp, i) for comp in gridElTemp])

In [39]:
# Example: Access ternaries_CompPos with their identifiers
for terncp, ternary_id in ternaries_with_id:
    print(f"Ternary composition position {terncp} has id {ternary_id}")

Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]) has id 0
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0]) has id 1
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]) has id 2
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 3
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 4
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 5
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 6
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 7
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 8
Ternary composition position ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 9


In [40]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    mapping[composition_key] = id

In [41]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    individual_elements = [elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary)))]
    mapping[composition_key] = {
        'id': id,
        'elements': individual_elements
    }

# Print the mapping to see the result
for key, value in mapping.items():
    print(f"Combination: {key}, ID: {value['id']}, Elements: {value['elements']}")

Combination: CrFeNi, ID: 0, Elements: ['Cr', 'Fe', 'Ni']
Combination: CrFeTi, ID: 1, Elements: ['Cr', 'Fe', 'Ti']
Combination: CrFeV, ID: 2, Elements: ['Cr', 'Fe', 'V']
Combination: CrNiTi, ID: 3, Elements: ['Cr', 'Ni', 'Ti']
Combination: CrNiV, ID: 4, Elements: ['Cr', 'Ni', 'V']
Combination: CrTiV, ID: 5, Elements: ['Cr', 'Ti', 'V']
Combination: FeNiTi, ID: 6, Elements: ['Fe', 'Ni', 'Ti']
Combination: FeNiV, ID: 7, Elements: ['Fe', 'Ni', 'V']
Combination: FeTiV, ID: 8, Elements: ['Fe', 'Ti', 'V']
Combination: NiTiV, ID: 9, Elements: ['Ni', 'Ti', 'V']


In [42]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [43]:
# def find_duplicates(lst):
#     duplicates_counts = {}
#     for item in lst:
#         # Use the string representation for complex/unhashable types
#         item_str = str(item)
#         if item_str in duplicates_counts:
#             duplicates_counts[item_str] += 1
#         else:
#             duplicates_counts[item_str] = 1

#     # Filter out items that only appear once
#     duplicates = {item: count for item, count in duplicates_counts.items() if count > 1}
    
#     if not duplicates:
#         return "No duplicates found in the list."
#     else:
#         result = "Duplicates found:"
#         for item, count in duplicates.items():
#             result += f"{item} appears {count} times"
#         return result.strip()

In [44]:
# find_duplicates(compositions)

In [45]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Cr', 'Fe'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('Cr', 'Ni'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('Fe', 'Ni'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('Cr', 'Ti'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('Fe', 'Ti'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('Cr', 'V'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('Fe', 'V'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('Ni', 'Ti'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('Ni', 'V'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('Ti', 'V'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [46]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [89, 88, 86, 83, 79, 74, 68, 61, 53]
Stitching 0 and 2 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 1 and 2 at ('Cr', 'Fe') from [89, 88, 86, 83, 79, 74, 68, 61, 53] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 0 and 3 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [179, 178, 176, 173, 169, 164, 158, 151, 143]
Stitching 0 and 4 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 3 and 4 at ('Cr', 'Ni') from [179, 178, 176, 173, 169, 164, 158, 151, 143] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 0 and 6 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [314, 313, 311, 308, 304, 299, 293, 286, 278]
Stitching 0 and 7 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [359, 358, 356, 353, 349, 344, 338, 331, 323]
Stitching 6 and 7 at ('Fe'

Stitching 8 and 9 at ('Ti', 'V') from [368, 367, 366, 365, 364, 363, 362, 361, 360] to [413, 412, 411, 410, 409, 408, 407, 406, 405]


In [47]:
from pycalphad import Database

# List of database file paths
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

Unique phases for ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB: ['LIQUID', 'B2_BCC', 'HCP_A3', 'HIGH_SIGMA', 'SIGMA', 'COV3', 'BCC_A2', 'L12_FCC', 'M3V', 'NI2V7', 'NI2V', 'FCC_A1']
Unique phases for ammap/databases/Cr-Fe-Ti_wang2017.tdb: ['LIQUID', 'C36', 'HCP_A3', 'BCC_B2', 'C14', 'SIGMA', 'BCC_A2', 'C15', 'TI5CR7FE17', 'FCC_A1']
Unique phases for ammap/databases/Cr-Fe-Ni_miettinen1999.tdb: ['LIQUID', 'HCP_A3', 'SIGMA', 'BCC_A2', 'FCC_A1']
Unique phases for ammap/databases/Cr-Ni-Ti_huang2018.tdb: ['LIQUID', 'NITI2', 'LAVES_C15', 'HCP_A3', 'NI3TI', 'LAVES_C36', 'LAVES_C14', 'NITI', 'BCC_A2', 'FCC_A1']
Unique phases for ammap/databases/Cr-Ti-V_ghosh2002.tdb: ['LIQUID', 'LAVES_C15', 'HCP_A3', 'LAVES_C36', 'LAVES_C14', 'BCC_A2']
Unique phases for ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb: ['LIQUID', 'A1', 'A2', 'FCC4', 'NI3TI', 'NITI2', 'C14', 'A3', 'BCC2']
Unique phases for ammap/databases/Fe-Ni-V_zhao2014.tdb: ['LIQUID', 'CBCC_A12', 'A2_BCC', 'FE4N', 'HCP_A3', 'A15_NI2V7', 'D022_NI3V', 'L

In [48]:

#dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')
#from myPycalphadCallable import equilibrium_callable

['Cr', 'Fe', 'Ni', 'Ti', 'V']
Loaded TDB file with phases considered: ['LIQUID', 'CBCC_A12', 'LAVES_C15', 'NITI2', 'HCP_A3', 'NI3TI', 'NIV3', 'LAVES_C14', 'DIAMOND_A4', 'B2', 'NI3V', 'SIGMA', 'BCC_A2', 'CUB_A13', 'NI2V', 'BCT_A5', 'FCC_A1']


In [49]:
import os
import importlib

# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/model.py:1278: UserWarning: The order-disorder model for "FCC4" has a contribution from the physical property model `magnetic_energy`. Partitioned physical properties are not correctly substituted into the disordered part of the energy. THE GIBBS ENERGY CALCULATED FOR THIS PHASE MAY BE INCORRECT. Please see the discussion in https://github.com/pycalphad/pycalphad/pull/311 for more details.
  warnings.warn(


Imported equilibrium_callable_CrFeTi_08bbfb9a: <function equilibrium_callable at 0x1529f92c9300>
Imported equilibrium_callable_CrFeV_ab1edb07: <function equilibrium_callable at 0x1529f11e7ba0>
Imported equilibrium_callable_CrNiTi_a9f6f2ff: <function equilibrium_callable at 0x1529f0c07e20>
Imported equilibrium_callable_NiTiV_1d83c99c: <function equilibrium_callable at 0x1529f0c6fba0>
Imported equilibrium_callable_NiCrV_b7aba9ab: <function equilibrium_callable at 0x1529f95ed4e0>
Imported equilibrium_callable_NiCrFe_f434a6d9: <function equilibrium_callable at 0x1529f01e9800>
Imported equilibrium_callable_FeTiV_fa95b3ee: <function equilibrium_callable at 0x1529f001c680>
Imported equilibrium_callable_FeNiV_b9b0384d: <function equilibrium_callable at 0x1529f0193d80>
Imported equilibrium_callable_CrTiV_ed4c332b: <function equilibrium_callable at 0x1529efa1bba0>
Imported equilibrium_callable_FeNiTi_06a49695: <function equilibrium_callable at 0x1529ef77a8e0>


In [50]:
# Get all files starting with "scheil"
scheil_files = [f for f in os.listdir(directory) if f.startswith("scheil") and f.endswith(".py")]
scheil_callables = {}

for file in scheil_files:
    module_name = file[:-3]
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    scheil_callables[callable_name] = getattr(module, "scheil_callable")

for name, func in scheil_callables.items():
    print(f"Imported {name}: {func}")


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/model.py:1278: UserWarning: The order-disorder model for "FCC4" has a contribution from the physical property model `magnetic_energy`. Partitioned physical properties are not correctly substituted into the disordered part of the energy. THE GIBBS ENERGY CALCULATED FOR THIS PHASE MAY BE INCORRECT. Please see the discussion in https://github.com/pycalphad/pycalphad/pull/311 for more details.
  warnings.warn(


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


Imported scheil_callable_FeNiTi_06a49695: <function scheil_callable at 0x1529ef77ade0>
Imported scheil_callable_NiCrFe_f434a6d9: <function scheil_callable at 0x1529ef849620>
Imported scheil_callable_CrNiTi_a9f6f2ff: <function scheil_callable at 0x1529ed7aa160>
Imported scheil_callable_FeNiV_b9b0384d: <function scheil_callable at 0x1529eda82700>
Imported scheil_callable_CrFeV_ab1edb07: <function scheil_callable at 0x1529eda83920>
Imported scheil_callable_NiCrV_b7aba9ab: <function scheil_callable at 0x1529ed206980>
Imported scheil_callable_FeTiV_fa95b3ee: <function scheil_callable at 0x1529ecadf100>
Imported scheil_callable_CrTiV_ed4c332b: <function scheil_callable at 0x1529ecadcd60>
Imported scheil_callable_NiTiV_1d83c99c: <function scheil_callable at 0x1529ecd422a0>
Imported scheil_callable_CrFeTi_08bbfb9a: <function scheil_callable at 0x1529ec53e5c0>


In [51]:
def process_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        middle_part = parts[2]
        if middle_part in element_mapping:
            return middle_part, element_mapping[middle_part]
    return None, None

In [52]:
element_mapping=mapping

In [53]:
def process_key(key):
    parts = key.split('_')
    if len(parts) >= 3:
        middle_part = parts[2]
        if middle_part in mapping:
            return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
        # If direct matching fails, try matching by elements
        middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
        for map_key, value in mapping.items():
            if set(value['elements']) == middle_elements:
                return middle_part, value['id'], value['elements']
    
    return None, None, None

# Process each key in the equilibrium_callables dictionary
id_to_callable = {}
for key in equilibrium_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

sc_id_to_callable = {}
for key in scheil_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        sc_id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

Key: equilibrium_callable_CrFeTi_08bbfb9a
The mapping number for CrFeTi is 1
Elements: ['Cr', 'Fe', 'Ti']
---
Key: equilibrium_callable_CrFeV_ab1edb07
The mapping number for CrFeV is 2
Elements: ['Cr', 'Fe', 'V']
---
Key: equilibrium_callable_CrNiTi_a9f6f2ff
The mapping number for CrNiTi is 3
Elements: ['Cr', 'Ni', 'Ti']
---
Key: equilibrium_callable_NiTiV_1d83c99c
The mapping number for NiTiV is 9
Elements: ['Ni', 'Ti', 'V']
---
Key: equilibrium_callable_NiCrV_b7aba9ab
The mapping number for NiCrV is 4
Elements: ['Cr', 'Ni', 'V']
---
Key: equilibrium_callable_NiCrFe_f434a6d9
The mapping number for NiCrFe is 0
Elements: ['Cr', 'Fe', 'Ni']
---
Key: equilibrium_callable_FeTiV_fa95b3ee
The mapping number for FeTiV is 8
Elements: ['Fe', 'Ti', 'V']
---
Key: equilibrium_callable_FeNiV_b9b0384d
The mapping number for FeNiV is 7
Elements: ['Fe', 'Ni', 'V']
---
Key: equilibrium_callable_CrTiV_ed4c332b
The mapping number for CrTiV is 5
Elements: ['Cr', 'Ti', 'V']
---
Key: equilibrium_callable_Fe

In [54]:
print(id_to_callable)
print(sc_id_to_callable)

{1: 'equilibrium_callable_CrFeTi_08bbfb9a', 2: 'equilibrium_callable_CrFeV_ab1edb07', 3: 'equilibrium_callable_CrNiTi_a9f6f2ff', 9: 'equilibrium_callable_NiTiV_1d83c99c', 4: 'equilibrium_callable_NiCrV_b7aba9ab', 0: 'equilibrium_callable_NiCrFe_f434a6d9', 8: 'equilibrium_callable_FeTiV_fa95b3ee', 7: 'equilibrium_callable_FeNiV_b9b0384d', 5: 'equilibrium_callable_CrTiV_ed4c332b', 6: 'equilibrium_callable_FeNiTi_06a49695'}
{6: 'scheil_callable_FeNiTi_06a49695', 0: 'scheil_callable_NiCrFe_f434a6d9', 3: 'scheil_callable_CrNiTi_a9f6f2ff', 7: 'scheil_callable_FeNiV_b9b0384d', 2: 'scheil_callable_CrFeV_ab1edb07', 4: 'scheil_callable_NiCrV_b7aba9ab', 8: 'scheil_callable_FeTiV_fa95b3ee', 5: 'scheil_callable_CrTiV_ed4c332b', 9: 'scheil_callable_NiTiV_1d83c99c', 1: 'scheil_callable_CrFeTi_08bbfb9a'}


In [55]:
compositions_with_id[:5]

[([0.0, 0.0, 1.0, 0.0, 0.0], 0),
 ([0.0, 0.125, 0.875, 0.0, 0.0], 0),
 ([0.0, 0.25, 0.75, 0.0, 0.0], 0),
 ([0.0, 0.375, 0.625, 0.0, 0.0], 0),
 ([0.0, 0.5, 0.5, 0.0, 0.0], 0)]

In [56]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [57]:
def reduce_compositions(compositions_with_id, mapping):
    element_order = elementalSpaceComponents
    system_comps_with_id = []
    reduced_compositions = []
    
    for composition, comp_id in compositions_with_id:
        relevant_entry = next((entry for entry in mapping.values() if entry['id'] == comp_id), None)
        if not relevant_entry:
            continue
        
        relevant_indices = [element_order.index(elem) for elem in relevant_entry['elements']]
        reduced_point = [composition[index] for index in relevant_indices]
        system_comps_with_id.append((reduced_point, comp_id))
        reduced_compositions.append(reduced_point)
    
    return system_comps_with_id, reduced_compositions

In [58]:
system_comps_with_id, reduced_compositions = reduce_compositions(compositions_with_id, mapping)

In [59]:
from tqdm.contrib.concurrent import process_map

In [60]:
startingNodes = [0, 90, 200, 310] + random.sample(range(len(reduced_compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {reduced_compositions[startingNode]}")

Starting nodes: [0, 90, 200, 310, 26, 137, 44, 393, 208, 136, 55, 429, 447, 19, 194]
Starting node: [0.0, 0.0, 1.0]
Starting node: [0.0, 0.0, 1.0]
Starting node: [0.25, 0.375, 0.375]
Starting node: [0.75, 0.125, 0.125]
Starting node: [0.375, 0.25, 0.375]
Starting node: [0.0, 0.25, 0.75]
Starting node: [1.0, 0.0, 0.0]
Starting node: [0.5, 0.375, 0.125]
Starting node: [0.375, 0.5, 0.125]
Starting node: [0.0, 0.125, 0.875]
Starting node: [0.125, 0.125, 0.75]
Starting node: [0.375, 0.0, 0.625]
Starting node: [0.875, 0.0, 0.125]
Starting node: [0.25, 0.25, 0.5]
Starting node: [0.125, 0.625, 0.25]


In [61]:
gridFeasible = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [62]:
# Function to get the correct callable for a given composition
def get_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_composition(elP):
    try:
        callable_func = get_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None


In [63]:
# Function to get the correct callable for a given composition
def get_sc_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = sc_id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_sc_composition(elP):
    try:
        callable_func = get_sc_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None

In [64]:
print(equilibrium_callables)
print(scheil_callables)

{'equilibrium_callable_CrFeTi_08bbfb9a': <function equilibrium_callable at 0x1529f92c9300>, 'equilibrium_callable_CrFeV_ab1edb07': <function equilibrium_callable at 0x1529f11e7ba0>, 'equilibrium_callable_CrNiTi_a9f6f2ff': <function equilibrium_callable at 0x1529f0c07e20>, 'equilibrium_callable_NiTiV_1d83c99c': <function equilibrium_callable at 0x1529f0c6fba0>, 'equilibrium_callable_NiCrV_b7aba9ab': <function equilibrium_callable at 0x1529f95ed4e0>, 'equilibrium_callable_NiCrFe_f434a6d9': <function equilibrium_callable at 0x1529f01e9800>, 'equilibrium_callable_FeTiV_fa95b3ee': <function equilibrium_callable at 0x1529f001c680>, 'equilibrium_callable_FeNiV_b9b0384d': <function equilibrium_callable at 0x1529f0193d80>, 'equilibrium_callable_CrTiV_ed4c332b': <function equilibrium_callable at 0x1529efa1bba0>, 'equilibrium_callable_FeNiTi_06a49695': <function equilibrium_callable at 0x1529ef77a8e0>}
{'scheil_callable_FeNiTi_06a49695': <function scheil_callable at 0x1529ef77ade0>, 'scheil_calla

In [65]:
sampled_compositions = compositions_with_id[::50]

# Print the sampled compositions to verify
for composition in sampled_compositions:
    print(composition)

([0.0, 0.0, 1.0, 0.0, 0.0], 0)
([0.0, 0.625, 0.0, 0.375, 0.0], 1)
([0.125, 0.125, 0.0, 0.0, 0.75], 2)
([0.125, 0.0, 0.75, 0.125, 0.0], 3)
([0.25, 0.0, 0.375, 0.0, 0.375], 4)
([0.375, 0.0, 0.0, 0.125, 0.5], 5)
([0.0, 0.5, 0.0, 0.5, 0.0], 6)
([0.0, 0.625, 0.0, 0.0, 0.375], 7)
([0.0, 0.75, 0.0, 0.125, 0.125], 8)


In [66]:
print(equilibrium_callables['equilibrium_callable_CrNiTi_a9f6f2ff'](compositions[200]))
compositions[200]

{'Phases': ['LAVES_C15', 'NITI'], 'PhaseFraction': [0.2751953798946548, 0.7248046201239813]}


[0.25, 0.0, 0.375, 0.0, 0.375]

In [67]:
from functools import partial

def get_equilibrium_callable(composition, id_to_callable, equilibrium_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = equilibrium_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_equilibrium_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [68]:
from functools import partial

def get_scheil_callable(composition, sc_id_to_callable, scheil_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = sc_id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = scheil_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_scheil_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

#WORKING
while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC'])) == 0 and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

In [69]:
import json

# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    #feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC','L12_FCC','BCC2', 'A1', 'A2', 'A3', 'FCC4'])) == 0 and p != [] for p in phases]
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('results.json', 'w') as f:
    json.dump(results_list, f, indent=4)

Queue: [0, 90, 200, 310, 26, 137, 44, 393, 208, 136, 55, 429, 447, 19, 194]
[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.25, 0.375, 0.375], [0.75, 0.125, 0.125], [0.375, 0.25, 0.375], [0.0, 0.25, 0.75], [1.0, 0.0, 0.0], [0.5, 0.375, 0.125], [0.375, 0.5, 0.125], [0.0, 0.125, 0.875], [0.125, 0.125, 0.75], [0.375, 0.0, 0.625], [0.875, 0.0, 0.125], [0.25, 0.25, 0.5], [0.125, 0.625, 0.25]]


0it [00:00, ?it/s]

Calculations done: 15    | Explored points: 31   
Queue: [1, 132, 9, 11, 12, 142, 18, 20, 25, 27, 31, 32, 42, 43, 181, 187, 444, 445, 449, 87, 91, 226, 99]
[[0.0, 0.125, 0.875], [0.875, 0.0, 0.125], [0.125, 0.0, 0.875], [0.125, 0.25, 0.625], [0.125, 0.375, 0.5], [0.0, 0.875, 0.125], [0.25, 0.125, 0.625], [0.25, 0.375, 0.375], [0.375, 0.125, 0.5], [0.375, 0.375, 0.25], [0.5, 0.125, 0.375], [0.5, 0.25, 0.25], [0.875, 0.0, 0.125], [0.875, 0.125, 0.0], [0.0, 0.125, 0.875], [0.0, 0.875, 0.125], [0.75, 0.0, 0.25], [0.75, 0.125, 0.125], [1.0, 0.0, 0.0], [0.875, 0.0, 0.125], [0.0, 0.125, 0.875], [0.0, 0.125, 0.875], [0.125, 0.0, 0.875]]


0it [00:00, ?it/s]

Calculations done: 38    | Explored points: 77   
Queue: [129, 2, 130, 3, 133, 4, 265, 10, 268, 269, 13, 17, 277, 150, 21, 24, 405, 285, 30, 415, 33, 35, 36, 37, 39, 40, 41, 175, 177, 186, 190, 322, 195, 325, 330, 85, 220, 92, 223, 227, 100, 361, 234, 235, 107, 370]
[[0.75, 0.0, 0.25], [0.0, 0.25, 0.75], [0.75, 0.125, 0.125], [0.0, 0.375, 0.625], [0.875, 0.125, 0.0], [0.0, 0.5, 0.5], [0.75, 0.125, 0.125], [0.125, 0.125, 0.75], [0.875, 0.125, 0.0], [1.0, 0.0, 0.0], [0.125, 0.5, 0.375], [0.25, 0.0, 0.75], [0.0, 0.875, 0.125], [0.125, 0.75, 0.125], [0.25, 0.5, 0.25], [0.375, 0.0, 0.625], [0.0, 0.0, 1.0], [0.125, 0.75, 0.125], [0.5, 0.0, 0.5], [0.125, 0.125, 0.75], [0.5, 0.375, 0.125], [0.625, 0.0, 0.375], [0.625, 0.125, 0.25], [0.625, 0.25, 0.125], [0.75, 0.0, 0.25], [0.75, 0.125, 0.125], [0.75, 0.25, 0.0], [0.75, 0.125, 0.125], [0.875, 0.0, 0.125], [0.0, 0.75, 0.25], [0.125, 0.125, 0.75], [0.0, 0.875, 0.125], [0.125, 0.75, 0.125], [0.125, 0.125, 0.75], [0.125, 0.75, 0.125], [0.75, 0.125,

...

Calculations done: 191   | Explored points: 275  
Queue: [96, 420, 232, 400, 240, 404, 375, 247]
[[0.0, 0.75, 0.25], [0.125, 0.75, 0.125], [0.0, 0.875, 0.125], [0.75, 0.125, 0.125], [0.125, 0.75, 0.125], [1.0, 0.0, 0.0], [0.125, 0.75, 0.125], [0.25, 0.625, 0.125]]


0it [00:00, ?it/s]

Calculations done: 199   | Explored points: 285  
Queue: [376, 233, 241, 248]
[[0.125, 0.875, 0.0], [0.0, 1.0, 0.0], [0.125, 0.875, 0.0], [0.25, 0.75, 0.0]]


0it [00:00, ?it/s]

Calculations done: 203   | Explored points: 291  


In [70]:

# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)

In [71]:
# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of scheil callables for each composition
    scheil_callables_list = [get_scheil_callable(system_comps_with_id[i], sc_id_to_callable, scheil_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_scheil_callable, zip(scheil_callables_list, elPositions), max_workers=4)
    else:
        results = [sc(elP) for sc, elP in zip(scheil_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['finalPhase'] for result in results]
    
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('scheil_results.json', 'w') as f:
    json.dump(results_list, f, indent=4)


Queue: [376, 233, 241, 248]
[[0.125, 0.875, 0.0], [0.0, 1.0, 0.0], [0.125, 0.875, 0.0], [0.25, 0.75, 0.0]]


0it [00:00, ?it/s]

Calculations done: 207   | Explored points: 291  
Queue: [254]
[[0.375, 0.625, 0.0]]


Calculations done: 208   | Explored points: 292  
Queue: [259, 253]
[[0.5, 0.5, 0.0], [0.375, 0.5, 0.125]]


Calculations done: 210   | Explored points: 294  
Queue: [258, 263]
[[0.5, 0.375, 0.125], [0.625, 0.375, 0.0]]


Calculations done: 212   | Explored points: 296  
Queue: [257, 266, 262]
[[0.5, 0.25, 0.25], [0.75, 0.25, 0.0], [0.625, 0.25, 0.125]]


Calculations done: 215   | Explored points: 299  


In [72]:

# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/scheil_results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_scheil_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)

Out of the explored points, around 1040 should be feasible (i.e., satisfy the equilibrium constraint).

In [73]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

In [74]:
pathFindEdges = []
for edge in edges:
    if gridFeasible[edge[0]] and gridFeasible[edge[1]]:
        pathFindEdges.append([edge[0], edge[1], 1])

In [75]:
pathfindingGraph = Graph(edges=pathFindEdges, bi_directional=False)

In [76]:
finder = DijkstraFinder()

In [77]:
len(list(pathfindingGraph.nodes.keys()))

178

In [78]:
print("Nodes in the graph:", list(pathfindingGraph.nodes.keys()))

Nodes in the graph: [0, 1, 9, 2, 10, 3, 11, 4, 12, 5, 13, 6, 14, 7, 15, 8, 16, 17, 18, 19, 20, 23, 24, 25, 26, 30, 31, 32, 35, 36, 37, 39, 40, 41, 42, 43, 44, 53, 61, 86, 88, 89, 90, 91, 99, 92, 100, 101, 97, 98, 106, 107, 108, 109, 114, 115, 120, 121, 125, 122, 126, 127, 129, 130, 131, 132, 133, 134, 143, 151, 158, 178, 179, 180, 189, 187, 188, 195, 196, 197, 202, 203, 204, 210, 215, 219, 222, 223, 224, 225, 226, 234, 227, 235, 228, 236, 229, 237, 230, 238, 231, 239, 232, 240, 233, 241, 242, 243, 244, 245, 248, 249, 250, 254, 255, 253, 258, 259, 260, 257, 262, 263, 264, 266, 267, 269, 278, 286, 293, 299, 304, 308, 311, 313, 314, 315, 324, 322, 323, 330, 331, 332, 329, 336, 337, 338, 342, 343, 344, 348, 349, 352, 353, 355, 356, 357, 358, 359, 360, 369, 362, 363, 364, 365, 366, 367, 368, 377, 407, 408, 409, 410, 411, 412, 413, 447, 449, 402]


In [79]:
# start_node_id = 194
# end_node_id = 830
start_node_id = 0#12
end_node_id = 408

# Check if the nodes exist in the graph
if start_node_id in pathfindingGraph.nodes and end_node_id in pathfindingGraph.nodes:
    path, runs = finder.find_path(
        pathfindingGraph.node(start_node_id), 
        pathfindingGraph.node(end_node_id), 
        pathfindingGraph)
    print("Path found:", path)
else:
    print(f"One or both of the nodes {start_node_id} and {end_node_id} do not exist in the graph.")

Path found: [<GraphNode(0 0x1529ea7c7ad0)>, <GraphNode(9 0x1529ebf10110)>, <GraphNode(17 0x1529eba38150)>, <GraphNode(24 0x1529eba38b10)>, <GraphNode(30 0x1529eba3b2d0)>, <GraphNode(35 0x1529eba391d0)>, <GraphNode(39 0x1529eba38fd0)>, <GraphNode(42 0x1529eba39690)>, <GraphNode(223 0x1529eac26550)>, <GraphNode(222 0x1529f9bd25d0)>, <GraphNode(219 0x1529f9bd1e50)>, <GraphNode(215 0x1529f9bd2490)>, <GraphNode(210 0x1529eba68750)>, <GraphNode(204 0x1529eba68810)>, <GraphNode(249 0x1529ebee5390)>, <GraphNode(243 0x1529ebee5610)>, <GraphNode(236 0x1529ebee5290)>, <GraphNode(228 0x1529ebee5190)>, <GraphNode(408 0x1529ebee70d0)>]


In [80]:
shortestPath = [p.node_id for p in path]

In [81]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(compositions):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# # Generate the projected grid
# gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y','z'])

# # Attach pure component (alloy) labels to corners
# pureComponentIndices = nimplex.pure_component_indexes_py(4, 12)
# labels = ['']*len(gridAtt_projected_df)
# for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
#     labels[idx] = "<b>"+comp+"</b>"

In [82]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")

 1: (  0) Ni100.0 
 2: (  9) Cr12.5 Ni87.5 
 3: ( 17) Cr25.0 Ni75.0 
 4: ( 24) Cr37.5 Ni62.5 
 5: ( 30) Cr50.0 Ni50.0 
 6: ( 35) Cr62.5 Ni37.5 
 7: ( 39) Cr75.0 Ni25.0 
 8: ( 42) Cr87.5 Ni12.5 
 9: (223) Cr87.5 Ni12.5 
10: (222) Cr87.5 V12.5 
11: (219) Cr75.0 V25.0 
12: (215) Cr62.5 V37.5 
13: (210) Cr50.0 V50.0 
14: (204) Cr37.5 V62.5 
15: (249) Cr37.5 V62.5 
16: (243) Cr25.0 Ti12.5 V62.5 
17: (236) Cr12.5 Ti25.0 V62.5 
18: (228) Ti37.5 V62.5 
19: (408) Ti37.5 V62.5 
